In [15]:
!pip install nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


import os

# Création d'un répertoire local au cas où les chemins par défaut posent problème
nltk_data_dir = os.path.join(os.getcwd(), "nltk_data")
os.makedirs(nltk_data_dir, exist_ok=True)

# Ajout manuel au chemin
nltk.data.path.append(nltk_data_dir)

# Téléchargements des ressources nécessaires dans le bon répertoire
nltk.download("punkt", download_dir=nltk_data_dir)
nltk.download("stopwords", download_dir=nltk_data_dir)
nltk.download("wordnet", download_dir=nltk_data_dir)




[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package punkt to /Users/salhinina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/salhinina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/salhinina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/salhinina/Desktop/Mach
[nltk_data]     ine_learning/Projet/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /Users/salhinina/Desktop/
[nltk_data]     Machine_learning/Projet/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/salhinina/Desktop/Ma
[nltk_data]     chine_learning/Projet/nltk_data...


True

In [29]:
df=pd.read_csv('scitweets_export.tsv', sep='\t')
display (df.head())

,Unnamed: 0,tweet_id,text,science_related,scientific_claim,scientific_reference,scientific_context
0,0,316669998137483264,Knees are a bit sore. i guess that's a sign th...,0,0.0,0.0,0.0
1,1,319090866545385472,McDonald's breakfast stop then the gym 🏀💪,0,0.0,0.0,0.0
2,2,322030931022065664,Can any Gynecologist with Cancer Experience ex...,1,1.0,0.0,0.0
3,3,322694830620807168,Couch-lock highs lead to sleeping in the couch...,1,1.0,0.0,0.0
4,4,328524426658328576,Does daily routine help prevent problems with ...,1,1.0,0.0,0.0


In [30]:
#-------------------------------
#----Analyse du jeu de données – partie scientifique uniquement--------
#------------------------------

In [34]:
# Filtrer uniquement les tweets "scientifiques"
df_sci = df[df["science_related"] == 1]


# Afficher la répartition des sous-labels
print(df_sci)
print("CLAIM :", df_sci["scientific_claim"].sum())
print("REF   :", df_sci["scientific_reference"].sum())
print("CONTEXT :", df_sci["scientific_context"].sum())


      Unnamed: 0             tweet_id  \
2              2   322030931022065664   
3              3   322694830620807168   
4              4   328524426658328576   
6              8   333266791960809472   
7              9   334282732085587968   
...          ...                  ...   
1128        1247  1336903053362954241   
1130        1249  1338450113765789699   
1131        1250  1338483958397480972   
1134        1253  1339398314563792900   
1135        1255  1340455669443350528   

                                                   text  science_related  \
2     Can any Gynecologist with Cancer Experience ex...                1   
3     Couch-lock highs lead to sleeping in the couch...                1   
4     Does daily routine help prevent problems with ...                1   
6     “Traffic Jam” In Brain’s Neurons Could Be Caus...                1   
7     Can playing more games improve lives and save ...                1   
...                                                

In [18]:
#-------------------------------
#----Formatage du jeu de données pour la tâche {CLAIM, REF} vs. CONTEXT--------
#------------------------------

In [35]:
# Garder uniquement les tweets avec CLAIM, REF, ou CONTEXT
df_sci = df_sci[
    (df_sci["scientific_claim"] == 1.0) |
    (df_sci["scientific_reference"] == 1.0) |
    (df_sci["scientific_context"] == 1.0)
]

# Supprimer les cas mixtes (contenant à la fois CLAIM/REF et CONTEXT)
df_sci = df_sci[~(
    (df_sci["scientific_context"] == 1.0) &
    ((df_sci["scientific_claim"] == 1.0) | (df_sci["scientific_reference"] == 1.0))
)]

# Créer la variable cible binaire : 1 = CLAIM ou REF, 0 = CONTEXT
df_sci["label"] = ((df_sci["scientific_claim"] == 1.0) | (df_sci["scientific_reference"] == 1.0)).astype(int)

# Afficher la distribution
df_sci["label"].value_counts()


label
1    124
0     33
Name: count, dtype: int64

In [ ]:
#-------------------------------
#----Nettoyage + normalisation du texte--------
#------------------------------

In [36]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Télécharger les ressources nécessaires
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(tokens)

# Appliquer le nettoyage
df_sci["clean_text"] = df_sci["text"].apply(clean_text)


[nltk_data] Downloading package punkt to /Users/salhinina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/salhinina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/salhinina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/salhinina/nltk_data'
    - '/Users/salhinina/Desktop/mon_env/nltk_data'
    - '/Users/salhinina/Desktop/mon_env/share/nltk_data'
    - '/Users/salhinina/Desktop/mon_env/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Users/salhinina/Desktop/Machine_learning/Projet/nltk_data'
**********************************************************************
